In [44]:
import pandas as pd
import numpy as np
import re
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
%matplotlib inline

In [45]:
file1 = r"C:\Users\Mr. Jarvis\Desktop\python project\project 3\counterfeit_train.csv"

In [46]:
file

'C:\\Users\\Mr. Jarvis\\Desktop\\python project\\project 3\\counterfeit_test.csv'

In [47]:
file2 = r"C:\Users\Mr. Jarvis\Desktop\python project\project 3\counterfeit_test.csv"

In [48]:
file2

'C:\\Users\\Mr. Jarvis\\Desktop\\python project\\project 3\\counterfeit_test.csv'

In [49]:
train = pd.read_csv(file1)

In [50]:
test = pd.read_csv(file2)

In [51]:
train.shape

(6818, 12)

In [52]:
test.shape

(1705, 11)

In [53]:
train.head()

,Medicine_ID,Counterfeit_Weight,DistArea_ID,Active_Since,Medicine_MRP,Medicine_Type,SidEffect_Level,Availability_rating,Area_Type,Area_City_Type,Area_dist_level,Counterfeit_Sales
0,RRA15,13.100,Area046,1995,160.2366,Antimalarial,critical,0.070422,DownTown,Tier 1,Small,1775.5026
1,YVV26,NaN,Area027,1983,110.4384,Mstablizers,mild,0.013000,CityLimits,Tier 3,Medium,3069.1520
2,LJC15,9.025,Area046,1995,259.4092,Cardiac,mild,0.060783,DownTown,Tier 1,Small,2603.0920
3,GWC40,11.800,Area046,1995,99.9830,OralContraceptives,mild,0.065555,DownTown,Tier 1,Small,1101.7130
4,QMN13,NaN,Area019,1983,56.4402,Hreplacements,critical,0.248859,MidTownResidential,Tier 1,Small,158.9402


In [54]:
test.head()

,Medicine_ID,Counterfeit_Weight,DistArea_ID,Active_Since,Medicine_MRP,Medicine_Type,SidEffect_Level,Availability_rating,Area_Type,Area_City_Type,Area_dist_level
0,HLZ81,NaN,Area027,1983,85.5328,Antibiotics,mild,0.112747,CityLimits,Tier 3,Medium
1,ECE94,13.45,Area045,2000,257.1460,OralContraceptives,mild,0.144446,DownTown,Tier 2,Unknown
2,SAD14,7.10,Area045,2000,98.1172,Antipyretics,mild,0.144221,DownTown,Tier 2,Unknown
3,EQV63,18.30,Area010,1996,135.3730,Tranquilizers,mild,0.100388,MidTownResidential,Tier 3,Unknown
4,AIR10,NaN,Area019,1983,112.8016,OralContraceptives,mild,0.022585,MidTownResidential,Tier 1,Small


In [55]:
train.dtypes

Medicine_ID             object
Counterfeit_Weight     float64
DistArea_ID             object
Active_Since             int64
Medicine_MRP           float64
Medicine_Type           object
SidEffect_Level         object
Availability_rating    float64
Area_Type               object
Area_City_Type          object
Area_dist_level         object
Counterfeit_Sales      float64
dtype: object

In [56]:
test.dtypes

Medicine_ID             object
Counterfeit_Weight     float64
DistArea_ID             object
Active_Since             int64
Medicine_MRP           float64
Medicine_Type           object
SidEffect_Level         object
Availability_rating    float64
Area_Type               object
Area_City_Type          object
Area_dist_level         object
dtype: object

In [57]:
train.isnull().sum()

Medicine_ID               0
Counterfeit_Weight     1166
DistArea_ID               0
Active_Since              0
Medicine_MRP              0
Medicine_Type             0
SidEffect_Level           0
Availability_rating       0
Area_Type                 0
Area_City_Type            0
Area_dist_level           0
Counterfeit_Sales         0
dtype: int64

In [58]:
test.isnull().sum()

Medicine_ID              0
Counterfeit_Weight     297
DistArea_ID              0
Active_Since             0
Medicine_MRP             0
Medicine_Type            0
SidEffect_Level          0
Availability_rating      0
Area_Type                0
Area_City_Type           0
Area_dist_level          0
dtype: int64

In [59]:
test['Counterfeit_Sales']=np.nan
test['data']='test'
train['data']='train'

In [60]:
train.shape

(6818, 13)

In [61]:
test.shape

(1705, 13)

In [62]:
all_data = pd.concat([train,test], axis=0)

In [63]:
all_data

,Medicine_ID,Counterfeit_Weight,DistArea_ID,Active_Since,Medicine_MRP,Medicine_Type,SidEffect_Level,Availability_rating,Area_Type,Area_City_Type,Area_dist_level,Counterfeit_Sales,data
0,RRA15,13.100,Area046,1995,160.2366,Antimalarial,critical,0.070422,DownTown,Tier 1,Small,1775.5026,train
1,YVV26,NaN,Area027,1983,110.4384,Mstablizers,mild,0.013000,CityLimits,Tier 3,Medium,3069.1520,train
2,LJC15,9.025,Area046,1995,259.4092,Cardiac,mild,0.060783,DownTown,Tier 1,Small,2603.0920,train
3,GWC40,11.800,Area046,1995,99.9830,OralContraceptives,mild,0.065555,DownTown,Tier 1,Small,1101.7130,train
4,QMN13,NaN,Area019,1983,56.4402,Hreplacements,critical,0.248859,MidTownResidential,Tier 1,Small,158.9402,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1700,KXW10,NaN,Area027,1983,136.5704,Hreplacements,mild,0.050505,CityLimits,Tier 3,Medium,NaN,test
1701,CKE54,21.300,Area035,2002,57.0744,Antibiotics,critical,0.041118,DownTown,Tier 2,Small,NaN,test
1702,HAY13,20.400,Area017,2005,182.7422,Antiseptics,mild,0.191273,DownTown,Tier 2,Unknown,NaN,test
1703,ZEE32,20.000,Area018,2007,266.9672,Hreplacements,mild,0.013000,Industrial,Tier 3,Medium,NaN,test


In [64]:
all_data.shape

(8523, 13)

In [65]:
all_data.dtypes

Medicine_ID             object
Counterfeit_Weight     float64
DistArea_ID             object
Active_Since             int64
Medicine_MRP           float64
Medicine_Type           object
SidEffect_Level         object
Availability_rating    float64
Area_Type               object
Area_City_Type          object
Area_dist_level         object
Counterfeit_Sales      float64
data                    object
dtype: object

In [66]:
all_data['Area_dist_level'].value_counts()

Medium     2793
Unknown    2410
Small      2388
High        932
Name: Area_dist_level, dtype: int64

In [67]:
all_data['Area_City_Type'].value_counts()

Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Area_City_Type, dtype: int64

In [68]:
all_data.shape

(8523, 13)

In [69]:
del all_data['Area_dist_level']

In [70]:
all_data.shape

(8523, 12)

In [71]:
cat_vars = all_data.select_dtypes(['object']).columns

In [72]:
cat_vars

Index(['Medicine_ID', 'DistArea_ID', 'Medicine_Type', 'SidEffect_Level',
       'Area_Type', 'Area_City_Type', 'data'],
      dtype='object')

In [73]:
for col in cat_vars[:-1]:
    dummy = pd.get_dummies(all_data[col],drop_first=True,prefix=col)
    all_data = pd.concat([all_data,dummy],axis=1)
    del all_data[col]
    print(col)
del dummy

Medicine_ID
DistArea_ID
Medicine_Type
SidEffect_Level
Area_Type
Area_City_Type


In [74]:
all_data.dtypes

Counterfeit_Weight              float64
Active_Since                      int64
Medicine_MRP                    float64
Availability_rating             float64
Counterfeit_Sales               float64
                                 ...   
Area_Type_DownTown                uint8
Area_Type_Industrial              uint8
Area_Type_MidTownResidential      uint8
Area_City_Type_Tier 2             uint8
Area_City_Type_Tier 3             uint8
Length: 1592, dtype: object

In [75]:
all_data.shape

(8523, 1592)

In [76]:
all_data["Counterfeit_Weight"].value_counts()

13.450    86
18.900    82
14.950    77
13.100    76
10.600    68
          ..
7.035      2
7.820      1
6.700      1
8.985      1
10.720     1
Name: Counterfeit_Weight, Length: 415, dtype: int64

In [77]:
all_data["Counterfeit_Weight"].isnull().sum()

1463

In [78]:
1166+297

1463

In [79]:
all_data['Counterfeit_Weight'].fillna(all_data['Counterfeit_Weight'].mean(), inplace=True)

In [80]:
all_data["Counterfeit_Weight"].isnull().sum()

0

In [81]:
all_data['Counterfeit_Sales'].fillna(all_data['Counterfeit_Sales'].mean(), inplace=True)

In [82]:
all_data['Counterfeit_Sales'].isnull().sum()

0

In [83]:
all_data.dtypes

Counterfeit_Weight              float64
Active_Since                      int64
Medicine_MRP                    float64
Availability_rating             float64
Counterfeit_Sales               float64
                                 ...   
Area_Type_DownTown                uint8
Area_Type_Industrial              uint8
Area_Type_MidTownResidential      uint8
Area_City_Type_Tier 2             uint8
Area_City_Type_Tier 3             uint8
Length: 1592, dtype: object

In [85]:
all_data.isnull().sum()

Counterfeit_Weight              0
Active_Since                    0
Medicine_MRP                    0
Availability_rating             0
Counterfeit_Sales               0
                               ..
Area_Type_DownTown              0
Area_Type_Industrial            0
Area_Type_MidTownResidential    0
Area_City_Type_Tier 2           0
Area_City_Type_Tier 3           0
Length: 1592, dtype: int64

In [86]:
all_data.isnull().sum()

Counterfeit_Weight              0
Active_Since                    0
Medicine_MRP                    0
Availability_rating             0
Counterfeit_Sales               0
                               ..
Area_Type_DownTown              0
Area_Type_Industrial            0
Area_Type_MidTownResidential    0
Area_City_Type_Tier 2           0
Area_City_Type_Tier 3           0
Length: 1592, dtype: int64

In [87]:
train = all_data[all_data['data']=='train']

In [88]:
test = all_data[all_data['data']=='test']

In [89]:
del train['data']

In [90]:
test.drop(['data','Counterfeit_Sales'],axis=1,inplace=True)

C:\Users\Mr. Jarvis\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [91]:
from sklearn.model_selection import cross_val_predict, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

In [92]:

from sklearn.model_selection import RandomizedSearchCV

In [93]:
x_train = train.drop(['Counterfeit_Sales'], axis=1)

In [94]:
x_train.shape

(6818, 1590)

In [95]:
y_train= train['Counterfeit_Sales']

In [96]:
y_train.shape

(6818,)

In [97]:
rfr = RandomForestRegressor( random_state = 42)

In [98]:
param_dist = {'max_depth': range(3,7),
            'n_estimators': (10, 50,15,20,25)}

In [99]:
n_iter_search=10

In [100]:
random_search = RandomizedSearchCV(rfr, param_distributions=param_dist,n_iter=n_iter_search,scoring='neg_mean_squared_error',cv=10,verbose=20)

In [101]:
?RandomizedSearchCV

In [102]:
random_search.fit(x_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] n_estimators=15, max_depth=5 ....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] . n_estimators=15, max_depth=5, score=-1040626.673, total=   2.1s
[CV] n_estimators=15, max_depth=5 ....................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.0s remaining:    0.0s


[CV] . n_estimators=15, max_depth=5, score=-1173818.307, total=   2.0s
[CV] n_estimators=15, max_depth=5 ....................................


...

[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:   53.6s remaining:    0.0s


[CV] . n_estimators=25, max_depth=6, score=-1140440.118, total=   3.5s
[CV] n_estimators=20, max_depth=3 ....................................
[CV] . n_estimators=20, max_depth=3, score=-1244222.393, total=   1.8s
[CV] n_estimators=20, max_depth=3 ....................................
[CV] . n_estimators=20, max_depth=3, score=-1346532.509, total=   1.8s
[CV] n_estimators=20, max_depth=3 ....................................
[CV] . n_estimators=20, max_depth=3, score=-1184179.286, total=   1.8s
[CV] n_estimators=20, max_depth=3 ....................................
[CV] . n_estimators=20, max_depth=3, score=-1312294.037, total=   1.9s
[CV] n_estimators=20, max_depth=3 ....................................
[CV] . n_estimators=20, max_depth=3, score=-1586103.901, total=   1.8s
[CV] n_estimators=20, max_depth=3 ....................................
[CV] . n_estimators=20, max_depth=3, score=-1331337.468, total=   1.8s
[CV] n_estimators=20, max_depth=3 ....................................
[CV] .

[CV] . n_estimators=50, max_depth=4, score=-1112190.069, total=   5.5s
[CV] n_estimators=50, max_depth=4 ....................................
[CV] . n_estimators=50, max_depth=4, score=-1261488.789, total=   5.6s
[CV] n_estimators=50, max_depth=4 ....................................
[CV] . n_estimators=50, max_depth=4, score=-1180562.612, total=   5.4s
[CV] n_estimators=20, max_depth=5 ....................................
[CV] . n_estimators=20, max_depth=5, score=-1040460.340, total=   2.6s
[CV] n_estimators=20, max_depth=5 ....................................
[CV] . n_estimators=20, max_depth=5, score=-1177702.956, total=   2.6s
[CV] n_estimators=20, max_depth=5 ....................................
[CV] . n_estimators=20, max_depth=5, score=-1125676.186, total=   2.7s
[CV] n_estimators=20, max_depth=5 ....................................
[CV] . n_estimators=20, max_depth=5, score=-1113995.690, total=   2.7s
[CV] n_estimators=20, max_depth=5 ....................................
[CV] .

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  4.3min finished


RandomizedSearchCV(cv=10, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   random_state=

In [103]:
random_search.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=6,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=25,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [104]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        # np.flatnonzero extracts index of `True` in a boolean array
        candidate = np.flatnonzero(results['rank_test_score'] == i)[0]
        # print rank of the model
        # values passed to function format here are put in the curly brackets when printing
        # 0 , 1 etc refer to placeholder for position of values passed to format function
        # .3f means upto 2 decimal digits
        print("Model with rank: {0}".format(i))
        # this prints cross validated performance and its standard deviation
        print("Mean validation score: {0:.5f} (std: {1:.5f})".format(
            results['mean_test_score'][candidate],
            results['std_test_score'][candidate]))
        # prints the paramter combination for which this performance was obtained
        print("Parameters: {0}".format(results['params'][candidate]))
        print("")

In [105]:
report(random_search.cv_results_,5)

Model with rank: 1
Mean validation score: -1147667.96562 (std: 95770.32871)
Parameters: {'n_estimators': 25, 'max_depth': 6}

Model with rank: 2
Mean validation score: -1147745.81514 (std: 97818.59340)
Parameters: {'n_estimators': 15, 'max_depth': 5}

Model with rank: 3
Mean validation score: -1148250.28637 (std: 96531.90387)
Parameters: {'n_estimators': 20, 'max_depth': 5}

Model with rank: 4
Mean validation score: -1149238.27064 (std: 97121.21618)
Parameters: {'n_estimators': 10, 'max_depth': 5}

Model with rank: 5
Mean validation score: -1168716.50114 (std: 93025.76551)
Parameters: {'n_estimators': 50, 'max_depth': 4}



In [145]:
rf=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=4,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [146]:
rf.fit(x_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=4,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
                      oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [147]:
test_score = rf.predict(test)

In [148]:
test_score

array([1616.48405032, 3997.14980618, 1751.63659549, ..., 2950.41913849,
       3997.14980618, 3976.92011696])

In [118]:
test3 = pd.read_csv(file2)

In [119]:
mid = test3['Medicine_ID']

In [120]:
mid = pd.DataFrame(mid)

In [121]:
Target = pd.DataFrame(test_score)

In [122]:
Target = Target.rename(columns={0:'Counterfeit_Sales'})

In [123]:
Final_Values = pd.concat([mid,Target],axis=1)

In [124]:
Final_Values

,Medicine_ID,Counterfeit_Sales
0,HLZ81,1616.484050
1,ECE94,3997.149806
2,SAD14,1751.636595
3,EQV63,382.329035
4,AIR10,374.012530
...,...,...
1700,KXW10,3101.551928
1701,CKE54,894.234016
1702,HAY13,2950.419138
1703,ZEE32,3997.149806


In [136]:
pd.DataFrame(Final_Values).to_csv("Nitesh_Bhosle_P3_p2.csv",index=False)